# Fixing Issues Related to Data (Loading)
**Learning how to deal with certain issues that can arise when using new experimental data.**

When loading experimental data into a reconstruction software like PtyPy, it is important that the geometry of the experiment is modelled correctly in the software and there are few things that can easily go wrong. The most common issue is that at least one of photon energy, detector distance or detector pixel size has been wrongly defined in the parameter tree. As part of the model initialistation in PtyPy, the reconstructed pixel size is being calculated based on these three physical quantities which might give the following error

```bash
RuntimeError: Arrays larger than 50M not supported. You requested 87706.44M pixels.
```

if they have not been provide correctly. Another common issue is that we often don't know exactely the coordinate system of the detector and the scanning stage that was used during data collection, and even if we know it might not match the default coordinate system used in PtyPy. Luckily, there are only a few (8 to be precise) possible configurations and the PtyScan base class has a parameter called [orientation](https://ptycho.github.io/ptypy/rst/parameters.html#scandata.PtyScan.orientation) that makes it very simple to do a brute-force search for the correct setup. 

Based on the small I08-1 data set that was used in the tutorial on [The HDF5 Loader](./6_the_hdf5_loader.ipynb) we have a created a new file named "dls_i08_nanogold_issue.h5" for which the coordinate system has been modified sligthly. The task for this tutorial is to figure out the correct ```orientation``` based on the data that has been provided. As a starting point, we set it to zero.

```python
p.scans.scan_00.data = u.Param()
p.scans.scan_00.data.name = 'Hdf5Loader'
p.scans.scan_00.data.orientation = 0
```

In addition, we assume that no meta information has been provided from the file and we need to set the photon energy, detector distance and detector pixel size directly

```python
p.scans.scan_00.data.energy = 810
p.scans.scan_00.data.distance = 0.072
p.scans.scan_00.data.psize = 44e-6
```

which unfortunately results in the RuntimeError mentioned above

```bash
RuntimeError: Arrays larger than 50M not supported. You requested 87706.44M pixels.
```

and even after we manage to fix this problem, the reconstruction does not look right

![](./_assets/i08_recons_dm_wrong.png)

<div class="alert alert-warning" markdown="1">
    <strong>Exersice</strong><br>Fix the typo when setting <code>energy</code>, <code>distance</code> or <code>psize</code> in the parameter tree (hint: check tutorial 6 with  <code>verbose_level="info"</code> for comparison) and find the correct <code>orientation</code>. Can you get the reconstruction to look as expected? 
</div>

---

In [ ]:
import ptypy, os
import ptypy.utils as u

# This will import the HDF5Loader class
ptypy.load_ptyscan_module("hdf5_loader")

# This will import the GPU engines
ptypy.load_gpu_engines("cuda")

# Root directory of tutorial data
tutorial_data_home = "../../data/"

# Dataset for this tutorial
dataset = "small_data/dls_i08_nanogold_issue.h5"

# Absolute path to HDF5 file with raw data
path_to_data = os.path.join(tutorial_data_home, dataset)

# Create parameter tree
p = u.Param()

# Set verbose level to interactive
p.verbose_level = "interactive"

# Set io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting during the reconstruction
p.io.autoplot = u.Param()
p.io.autoplot.active=True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 10

# Define the scan model
p.scans = u.Param()
p.scans.scan_00 = u.Param()
p.scans.scan_00.name = 'Full'

# Initial illumination (based on simulated optics)
p.scans.scan_00.illumination = u.Param()
p.scans.scan_00.illumination.model = None
p.scans.scan_00.illumination.photons = None
p.scans.scan_00.illumination.aperture = u.Param()
p.scans.scan_00.illumination.aperture.form = "circ"
p.scans.scan_00.illumination.aperture.size = 333e-6
p.scans.scan_00.illumination.propagation = u.Param()
p.scans.scan_00.illumination.propagation.focussed = 13.725e-3
p.scans.scan_00.illumination.propagation.parallel = 45e-6

# Data loader
p.scans.scan_00.data = u.Param()
p.scans.scan_00.data.name = 'Hdf5Loader'
p.scans.scan_00.data.orientation = 0

# Read diffraction data
p.scans.scan_00.data.intensities = u.Param()
p.scans.scan_00.data.intensities.file = path_to_data
p.scans.scan_00.data.intensities.key = "data"

# Read positions data
p.scans.scan_00.data.positions = u.Param()
p.scans.scan_00.data.positions.file = path_to_data
p.scans.scan_00.data.positions.slow_key = "posy_mm"
p.scans.scan_00.data.positions.slow_multiplier = 1e-3
p.scans.scan_00.data.positions.fast_key = "posx_mm"
p.scans.scan_00.data.positions.fast_multiplier = 1e-3

# Provide meta information
p.scans.scan_00.data.energy = 810
p.scans.scan_00.data.distance = 0.072
p.scans.scan_00.data.psize = 44e-6

# Define reconstruction engine (using DM)
p.engines = u.Param()
p.engines.engine = u.Param()
p.engines.engine.name = "DM_pycuda"
p.engines.engine.numiter = 200
p.engines.engine.numiter_contiguous = 1
p.engines.engine.alpha = 0.9
p.engines.engine.probe_support = None
p.engines.engine.probe_update_start = 0

# Run reconstruction
P = ptypy.core.Ptycho(p,level=5)